# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [15]:
import sys; sys.path.insert(0, '..')
import itertools

In [16]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [17]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [18]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [19]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [20]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [21]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [22]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [23]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [24]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [25]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [26]:
main_node = "run"
#delete_main_node(main_node)
#add_main_node(main_node)

### Set up the query

query = ''

result = topic_concordancer.main(query,window=50,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  

In [27]:
lemmas = ['scream','run']

In [28]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="scream"][]{0,25}[lemma="run"])|([lemma="run"][]{0,25}[lemma="scream"])


In [29]:
domain_term = "scream"

In [30]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [31]:
fragment_1 = {}
fragment_1['original_sentence'] = "You know, of people running and screaming and falling and the schnell-schnell-schnell you know and this is incredible."
fragment_1['label']="(..) people running and screaming and falling and the schnell-schnell-schnell (..)."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22of%22%5D+%5B%22people%22%5D+%5B%22running%22%5D+%5B%22and%22%5D+%5B%22screaming%22%5D+%5B%22and%22%5D+%5B%22falling%22%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%5D%7B0%2C3%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%22and%22%5D+%5B%22this%22%5D+%5B%22is%22%5D+%5B%22incredible%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You know , of people running and screaming and falling and the schnell-schnell-schnell you know and this is incredible . ', 'right': '', 'complete_match': 'You know , of people running and screaming and falling and the schnell-schnell-schnell you know and this is incredible . ', 'testimony_id': 'HVT-78', 'shelfmark': ['Fortunoff Archive HVT-78'], 'token_start': 16132, 'token_end': 16152}


In [32]:
fragment_2 = {}
fragment_2['original_sentence'] = "Cause I'm screaming, sometimes running, screaming and standing by the crematorium."
fragment_2['label']="Cause I'm screaming, sometimes running, screaming and standing by the crematorium."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Cause%22%5D+%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22screaming%22%5D+%5B%5D+%5B%22sometimes%22%5D+%5B%22running%22%5D+%5B%5D+%5B%22screaming%22%5D+%5B%22and%22%5D+%5B%22standing%22%5D+%5B%22by%22%5D+%5B%22the%22%5D+%5B%22crematorium%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "Cause I 'm screaming , sometimes running , screaming and standing by the crematorium . ", 'right': '', 'complete_match': "Cause I 'm screaming , sometimes running , screaming and standing by the crematorium . ", 'testimony_id': 'irn504669', 'shelfmark': ['USHMM RG-50.030*0166'], 'token_start': 5818, 'token_end': 5833}


In [33]:
fragment_3 = {}
fragment_3['original_sentence'] = "I'm crying and screaming and-- and running, wondering if-- if I'm going to be shot while I'm running."
fragment_3['label']="I'm crying and screaming and-- and running, wondering if-- if I'm going to be shot while I'm running."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22crying%22%5D+%5B%22and%22%5D+%5B%22screaming%22%5D+%5B%22and%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22running%22%5D+%5B%5D+%5B%22wondering%22%5D+%5B%22if%22%5D+%5B%5D%7B0%2C3%7D+%5B%22if%22%5D+%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22be%22%5D+%5B%22shot%22%5D+%5B%22while%22%5D+%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22running%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I 'm crying and screaming and -- and running , wondering if -- if I 'm going to be shot while I 'm running . ", 'right': '', 'complete_match': "I 'm crying and screaming and -- and running , wondering if -- if I 'm going to be shot while I 'm running . ", 'testimony_id': 'HVT-16', 'shelfmark': ['Fortunoff Archive HVT-16'], 'token_start': 3379, 'token_end': 3404}


In [34]:
fragment_4 = {}
fragment_4['original_sentence'] = "People running. People screaming."
fragment_4['label']= "People running. People screaming."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22People%22%5D+%5B%22running%22%5D+%5B%5D+%5B%22People%22%5D+%5B%22screaming%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'People running . People screaming . ', 'right': '', 'complete_match': 'People running . People screaming . ', 'testimony_id': 'usc_shoah_15732', 'shelfmark': ['USC Shoah Foundation 15732'], 'token_start': 11242, 'token_end': 11248}


In [35]:
fragment_5 = {}
fragment_5['original_sentence'] = "They were shooting and they were running and they were screaming."
fragment_5['label']= "They were shooting and they were running and they were screaming."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22were%22%5D+%5B%22shooting%22%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22running%22%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22screaming%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They were shooting and they were running and they were screaming . ', 'right': '', 'complete_match': 'They were shooting and they were running and they were screaming . ', 'testimony_id': 'irn508740', 'shelfmark': ['USHMM RG-50.462*0099'], 'token_start': 4554, 'token_end': 4566}


In [36]:
add_testimonial_fragments(fragments)